# Let us continue - This is Part 2

#### TODO:

Import pandas as pd and read the csv file again

In [2]:
import pandas as pd

In [3]:
covid_df = pd.read_csv('italy-covid-daywise.csv')

In [4]:
covid_df

,date,new_cases,new_deaths,new_tests
0,2020-02-22,14.0,0.0,NaN
1,2020-02-23,62.0,2.0,NaN
2,2020-02-24,53.0,0.0,NaN
3,2020-02-25,97.0,4.0,NaN
4,2020-02-26,93.0,5.0,NaN
...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0
191,2020-08-31,1365.0,4.0,42583.0
192,2020-09-01,996.0,6.0,54395.0
193,2020-09-02,975.0,8.0,NaN


#### TODO: From previous part

sort_values() of 'new_cases' and print the first 10 rows. You don't need to set the parameter ascending

In [5]:
covid_df.sort_values('new_cases').head()

,date,new_cases,new_deaths,new_tests
119,2020-06-20,-148.0,47.0,29875.0
0,2020-02-22,14.0,0.0,NaN
2,2020-02-24,53.0,0.0,NaN
1,2020-02-23,62.0,2.0,NaN
5,2020-02-27,78.0,1.0,NaN


It seems like the count of new cases on Jun 20, 2020, was `-148`, a negative number! Not something we might have expected, but that's the nature of real-world data. It could be a data entry error, or the government may have issued a correction to account for miscounting in the past. Can you dig through news articles online and figure out why the number was negative?

#### TODO:

Let's look at some days before and after Jun 20, 2020.

In [6]:
covid_df[115:125]

,date,new_cases,new_deaths,new_tests
115,2020-06-16,301.0,26.0,27762.0
116,2020-06-17,210.0,34.0,33957.0
117,2020-06-18,328.0,43.0,32921.0
118,2020-06-19,331.0,66.0,28570.0
119,2020-06-20,-148.0,47.0,29875.0
120,2020-06-21,264.0,49.0,24581.0
121,2020-06-22,224.0,24.0,16152.0
122,2020-06-23,221.0,23.0,23225.0
123,2020-06-24,113.0,18.0,30237.0
124,2020-06-25,577.0,-31.0,29421.0


For now, let's assume this was indeed a data entry error. We can use one of the following approaches for dealing with the faulty value:
1. Replace it with `0`.
2. Replace it with the average of the entire column
3. Replace it with the average of the values on the previous & next date //More or less best practice in this particular case
4. Discard the row entirely

#### TODO:

Which approach you pick requires some context about the data and the problem. In this case, since we are dealing with data ordered by date, we can go ahead with the third approach.

You can use the `.at` method to modify a specific value within the dataframe.

In [7]:
covid_df.at[119, 'new_cases'] = (covid_df.at[118, 'new_cases'] + covid_df.at[120, 'new_cases']) / 2 # Option 3 of list above

In [8]:
covid_df.at[119, 'new_cases'] = covid_df.new_cases.mean() # Option 2 of above list (bad idea in this case)

In [9]:
covid_df[115:125]

,date,new_cases,new_deaths,new_tests
115,2020-06-16,301.000000,26.0,27762.0
116,2020-06-17,210.000000,34.0,33957.0
117,2020-06-18,328.000000,43.0,32921.0
118,2020-06-19,331.000000,66.0,28570.0
119,2020-06-20,1394.653846,47.0,29875.0
120,2020-06-21,264.000000,49.0,24581.0
121,2020-06-22,224.000000,24.0,16152.0
122,2020-06-23,221.000000,23.0,23225.0
123,2020-06-24,113.000000,18.0,30237.0
124,2020-06-25,577.000000,-31.0,29421.0


## Missing data

As data comes in many shapes and forms, pandas aims to be flexible with regard to handling missing data. While `NaN` is the default missing value marker for reasons of computational speed and convenience, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object.

#### TODO:

Get the total of all NaN values in every column

In [10]:
covid_df.isnull().sum() #isnull() searches for NaN, .sum() adds them all up

date           0
new_cases      0
new_deaths     0
new_tests     60
dtype: int64

We can use one of the following approaches for dealing with the missing value:
1. Given a large dataset, if a column contains  more than 15% missing values, delete the corresponding column
2. Given a large dataset, if there is only very few observation, delete the observation and keep the column
3. Replace the missing values with the following aggregate values: Mean, Mode, Median, constant Value (U)

#### TODO

Drop all NaN. set inplace to False. Set inplace to True and see the difference

In [11]:
covid_df.dropna(inplace=False) #dropna is a permanent change to the dataframe CANNOT UNDO
#inplace=True will restart the index at zero and go from there

,date,new_cases,new_deaths,new_tests
58,2020-04-20,3047.0,433.0,7841.0
59,2020-04-21,2256.0,454.0,28095.0
60,2020-04-22,2729.0,534.0,44248.0
61,2020-04-23,3370.0,437.0,37083.0
62,2020-04-24,2646.0,464.0,95273.0
...,...,...,...,...
188,2020-08-28,1409.0,5.0,65135.0
189,2020-08-29,1460.0,9.0,64294.0
190,2020-08-30,1444.0,1.0,53541.0
191,2020-08-31,1365.0,4.0,42583.0


https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

## Working with dates

While we've looked at overall numbers for the cases, tests, positive rate, etc., it would also be useful to study these numbers on a month-by-month basis. The `date` column might come in handy here, as Pandas provides many utilities for working with dates.

#### TODO:

Check the columns' data types

In [12]:
covid_df.dtypes

date           object
new_cases     float64
new_deaths    float64
new_tests     float64
dtype: object

#### TODO:

access the date column

In [13]:
covid_df.date

0      2020-02-22
1      2020-02-23
2      2020-02-24
3      2020-02-25
4      2020-02-26
          ...    
190    2020-08-30
191    2020-08-31
192    2020-09-01
193    2020-09-02
194    2020-09-03
Name: date, Length: 195, dtype: object

The data type of date is currently `object`, so Pandas does not know that this column is a date. We can convert it into a `datetime` column using the `pd.to_datetime` method.

#### TODO:

Convert the date column to datetime date type

In [14]:
covid_df.date = pd.to_datetime(covid_df.date)

In [15]:
covid_df.date

0     2020-02-22
1     2020-02-23
2     2020-02-24
3     2020-02-25
4     2020-02-26
         ...    
190   2020-08-30
191   2020-08-31
192   2020-09-01
193   2020-09-02
194   2020-09-03
Name: date, Length: 195, dtype: datetime64[ns]

You can see that it now has the datatype `datetime64`. We can now extract different parts of the data into separate columns, using the `DatetimeIndex` class ([view docs](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DatetimeIndex.html)).

#### TODO:

Create 4 new columns.

1. year column
2. month column
3. day column
4. weekday column

In [16]:
covid_df['year'] = pd.DatetimeIndex(covid_df.date).year
covid_df['year'] = covid_df.date.dt.year #Does the same thing, this is simpler
covid_df

,date,new_cases,new_deaths,new_tests,year
0,2020-02-22,14.0,0.0,NaN,2020
1,2020-02-23,62.0,2.0,NaN,2020
2,2020-02-24,53.0,0.0,NaN,2020
3,2020-02-25,97.0,4.0,NaN,2020
4,2020-02-26,93.0,5.0,NaN,2020
...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020
191,2020-08-31,1365.0,4.0,42583.0,2020
192,2020-09-01,996.0,6.0,54395.0,2020
193,2020-09-02,975.0,8.0,NaN,2020


In [17]:
covid_df['month'] = covid_df.date.dt.month
covid_df

,date,new_cases,new_deaths,new_tests,year,month
0,2020-02-22,14.0,0.0,NaN,2020,2
1,2020-02-23,62.0,2.0,NaN,2020,2
2,2020-02-24,53.0,0.0,NaN,2020,2
3,2020-02-25,97.0,4.0,NaN,2020,2
4,2020-02-26,93.0,5.0,NaN,2020,2
...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8
191,2020-08-31,1365.0,4.0,42583.0,2020,8
192,2020-09-01,996.0,6.0,54395.0,2020,9
193,2020-09-02,975.0,8.0,NaN,2020,9


In [18]:
covid_df['day'] = covid_df.date.dt.day
covid_df

,date,new_cases,new_deaths,new_tests,year,month,day
0,2020-02-22,14.0,0.0,NaN,2020,2,22
1,2020-02-23,62.0,2.0,NaN,2020,2,23
2,2020-02-24,53.0,0.0,NaN,2020,2,24
3,2020-02-25,97.0,4.0,NaN,2020,2,25
4,2020-02-26,93.0,5.0,NaN,2020,2,26
...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31
192,2020-09-01,996.0,6.0,54395.0,2020,9,1
193,2020-09-02,975.0,8.0,NaN,2020,9,2


In [19]:
covid_df['weekday'] = covid_df.date.dt.weekday
covid_df

,date,new_cases,new_deaths,new_tests,year,month,day,weekday
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2
...,...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30,6
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31,0
192,2020-09-01,996.0,6.0,54395.0,2020,9,1,1
193,2020-09-02,975.0,8.0,NaN,2020,9,2,2


Let's check the overall metrics for May. We can query the rows for May, choose a subset of columns, and use the `sum` method to aggregate each selected column's values.

#### TODO:

Query the rows for May

In [20]:
covid_df_may = covid_df[covid_df.month == 5]
covid_df_may

,date,new_cases,new_deaths,new_tests,year,month,day,weekday
69,2020-05-01,1872.0,285.0,43732.0,2020,5,1,4
70,2020-05-02,1965.0,269.0,31231.0,2020,5,2,5
71,2020-05-03,1900.0,474.0,27047.0,2020,5,3,6
72,2020-05-04,1389.0,174.0,22999.0,2020,5,4,0
73,2020-05-05,1221.0,195.0,32211.0,2020,5,5,1
74,2020-05-06,1075.0,236.0,37771.0,2020,5,6,2
75,2020-05-07,1444.0,369.0,13665.0,2020,5,7,3
76,2020-05-08,1401.0,274.0,45428.0,2020,5,8,4
77,2020-05-09,1327.0,243.0,36091.0,2020,5,9,5
78,2020-05-10,1083.0,194.0,31384.0,2020,5,10,6


#### TODO:

Extract the subset of columns (new_cases, new_deaths, new_tests) during the month of may to perform aggregation on them later

In [21]:
covid_df_may_metrics = covid_df_may[['new_cases', 'new_deaths', 'new_tests']]
covid_df_may_metrics

,new_cases,new_deaths,new_tests
69,1872.0,285.0,43732.0
70,1965.0,269.0,31231.0
71,1900.0,474.0,27047.0
72,1389.0,174.0,22999.0
73,1221.0,195.0,32211.0
74,1075.0,236.0,37771.0
75,1444.0,369.0,13665.0
76,1401.0,274.0,45428.0
77,1327.0,243.0,36091.0
78,1083.0,194.0,31384.0


### TODO:

Sum the values in the above-mentioned columns. Store the results into a variable called covid_may_totals

In [22]:
covid_df_may_totals = covid_df_may_metrics.sum()
covid_df_may_totals

new_cases       29073.0
new_deaths       5658.0
new_tests     1078720.0
dtype: float64

### TODO:

Check the type of the variable covid_may_totals

In [23]:
type(covid_df_may_totals)

pandas.core.series.Series

#### TODO:

Combine the above operations into a single statement.

In [24]:
covid_df.loc[covid_df.month == 5][['new_cases', 'new_deaths', 'new_tests']].sum()

new_cases       29073.0
new_deaths       5658.0
new_tests     1078720.0
dtype: float64

As another example, let's check if the number of cases reported on Sundays is higher than the average number of cases reported every day. This time, we might want to aggregate columns using the `.mean` method.

#### TODO:

Get the average of new_cases at every day (The overall average)

In [25]:
covid_df.new_cases.mean()

1400.2802761341225

#### TODO:

Get the average of new_cases on Sunday

In [26]:
covid_df.loc[covid_df.weekday == 6].new_cases.mean()

1559.0714285714287

It seems like more cases were reported on Sundays compared to other days.

## Grouping and aggregation

As a next step, we might want to summarize the day-wise data and create a new dataframe with month-wise data. We can use the `groupby` function to create a group for each month, select the columns we wish to aggregate, and aggregate them using the `sum` method. 

#### TODO:

Group new_cases, new_deaths, and new_tests by month

In [27]:
covid_month_df = covid_df.groupby('month')[['new_cases', 'new_deaths','new_tests']].sum()

In [28]:
covid_month_df

,new_cases,new_deaths,new_tests
month,,,
2,885.000000,21.0,0.0
3,100851.000000,11570.0,0.0
4,101852.000000,16091.0,419591.0
5,29073.000000,5658.0,1078720.0
6,9314.653846,1404.0,830354.0
7,6722.000000,388.0,797692.0
8,21060.000000,345.0,1098704.0
9,3297.000000,20.0,54395.0


The result is a new data frame that uses unique values from the column passed to `groupby` as the index. Grouping and aggregation is a powerful method for progressively summarizing data into smaller data frames.

Instead of aggregating by sum, you can also aggregate by other measures like mean. 

#### TODO:

Let's compute the average number of daily new cases, deaths, and tests for each month.

In [29]:
covid_daily_df = covid_df.groupby('day')[['new_cases', 'new_deaths','new_tests']].mean()

In [30]:
covid_daily_df

,new_cases,new_deaths,new_tests
day,,,
1,1148.142857,177.857143,35482.00
2,1276.857143,156.571429,27625.50
3,1285.428571,192.857143,21557.25
4,1190.500000,177.333333,25721.75
5,1202.500000,169.500000,30896.50
6,1209.000000,151.000000,29436.75
7,1116.666667,190.000000,23610.25
8,1095.500000,166.666667,29259.75
9,1245.833333,168.166667,30252.75


Apart from grouping, another form of aggregation is the running or cumulative sum of cases, tests, or death up to each row's date. We can use the `cumsum` method to compute the cumulative sum of a column as a new series. Let's add three new columns: `total_cases`, `total_deaths`, and `total_tests`.

#### TODO:

Do the cumulative sum for new_cases, new_deaths, and new_tests and store the values into three new columns (total_cases, total_deaths, and total_tests)

In [31]:
# Remember that the initial tests are the following

initial_tests = 935310

In [32]:
covid_df

,date,new_cases,new_deaths,new_tests,year,month,day,weekday
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2
...,...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30,6
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31,0
192,2020-09-01,996.0,6.0,54395.0,2020,9,1,1
193,2020-09-02,975.0,8.0,NaN,2020,9,2,2


In [33]:
covid_df['total_cases'] = covid_df.new_cases.cumsum()

In [34]:
covid_df['total_deaths'] = covid_df.new_deaths.cumsum()

In [35]:
covid_df['total_tests'] = covid_df.new_tests.cumsum()

In [36]:
covid_df

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.000000,0.0,NaN
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.000000,2.0,NaN
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.000000,2.0,NaN
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.000000,6.0,NaN
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.000000,11.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30,6,268392.653846,35473.0,4182478.0
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31,0,269757.653846,35477.0,4225061.0
192,2020-09-01,996.0,6.0,54395.0,2020,9,1,1,270753.653846,35483.0,4279456.0
193,2020-09-02,975.0,8.0,NaN,2020,9,2,2,271728.653846,35491.0,NaN


Notice how the `NaN` values in the `total_tests` column remain unaffected.

## Merging data from multiple sources

To determine other metrics like test per million, cases per million, etc., we require some more information about the country, viz. its population. Let's open another file `locations.csv` that contains health-related information for many countries, including Italy.

#### TODO:

Read the locations.csv file into a new dataframe and call it locations_df

In [37]:
locations_df = pd.read_csv('locations.csv')

In [38]:
locations_df.head()

,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita
0,Afghanistan,Asia,38928341.0,64.83,0.50,1803.987
1,Albania,Europe,2877800.0,78.57,2.89,11803.431
2,Algeria,Africa,43851043.0,76.88,1.90,13913.839
3,Andorra,Europe,77265.0,83.73,NaN,NaN
4,Angola,Africa,32866268.0,61.15,NaN,5819.495


#### TODO:

Find the entry for location=='Italy'

In [39]:
covid_df.head()

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.0,0.0,NaN
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.0,2.0,NaN
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.0,2.0,NaN
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.0,6.0,NaN
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.0,11.0,NaN


In [40]:
locations_df.loc[locations_df.location == 'Italy']

,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita
97,Italy,Europe,60461828.0,83.51,3.18,35220.084


In [41]:
covid_df['location'] = 'Italy'

In [55]:
covid_df.head()

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.0,0.0,NaN,Italy
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.0,2.0,NaN,Italy
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.0,2.0,NaN,Italy
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.0,6.0,NaN,Italy
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.0,11.0,NaN,Italy


We can merge this data into our existing data frame by adding more columns. However, to merge two data frames, we need at least one common column. Let's insert a `location` column in the `covid_df` dataframe with all values set to `"Italy"`.

#### TODO:

Create a new column in the covid_df dataframe called 'location' and set the value to 'Italy'

.merge will merge the dataframes

In [43]:
merged_df = covid_df.merge(locations_df, on='location')

In [44]:
merged_df

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.000000,0.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.000000,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.000000,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.000000,6.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.000000,11.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30,6,268392.653846,35473.0,4182478.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31,0,269757.653846,35477.0,4225061.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
192,2020-09-01,996.0,6.0,54395.0,2020,9,1,1,270753.653846,35483.0,4279456.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
193,2020-09-02,975.0,8.0,NaN,2020,9,2,2,271728.653846,35491.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084


We can now add the columns from `locations_df` into `covid_df` using the `.merge` method.

#### TODO:

Merge covid_df and locations_df on='location' into a new dataframe. Call the new dataframe merged_df

In [45]:
merged_df

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.000000,0.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.000000,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.000000,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.000000,6.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.000000,11.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2020-08-30,1444.0,1.0,53541.0,2020,8,30,6,268392.653846,35473.0,4182478.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
191,2020-08-31,1365.0,4.0,42583.0,2020,8,31,0,269757.653846,35477.0,4225061.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
192,2020-09-01,996.0,6.0,54395.0,2020,9,1,1,270753.653846,35483.0,4279456.0,Italy,Europe,60461828.0,83.51,3.18,35220.084
193,2020-09-02,975.0,8.0,NaN,2020,9,2,2,271728.653846,35491.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084


The location data for Italy is appended to each row within `covid_df`. If the `covid_df` data frame contained data for multiple locations, then the respective country's location data would be appended for each row.

We can now calculate metrics like cases per million, deaths per million, and tests per million.

### TODO:

Calculate the cases per million

In [46]:
merged_df['cases_per_million'] = merged_df.new_cases * 1e6 / merged_df.population

In [47]:
merged_df.head()

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita,cases_per_million
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.0,0.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.231551
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.025440
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.876586
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.0,6.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.604318
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.0,11.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.538161


### TODO:

Calculate the deaths per million

In [48]:
merged_df['deaths_per_million'] = merged_df.new_deaths * 1e6 / merged_df.population

In [49]:
merged_df.head()

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita,cases_per_million,deaths_per_million
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.0,0.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.231551,0.000000
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.025440,0.033079
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.876586,0.000000
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.0,6.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.604318,0.066157
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.0,11.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.538161,0.082697


### TODO:

Calculate the tests per million

In [50]:
merged_df['tests_per_million'] = merged_df.new_tests * 1e6 / merged_df.population

In [51]:
merged_df.head()

,date,new_cases,new_deaths,new_tests,year,month,day,weekday,total_cases,total_deaths,total_tests,location,continent,population,life_expectancy,hospital_beds_per_thousand,gdp_per_capita,cases_per_million,deaths_per_million,tests_per_million
0,2020-02-22,14.0,0.0,NaN,2020,2,22,5,14.0,0.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.231551,0.000000,NaN
1,2020-02-23,62.0,2.0,NaN,2020,2,23,6,76.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.025440,0.033079,NaN
2,2020-02-24,53.0,0.0,NaN,2020,2,24,0,129.0,2.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,0.876586,0.000000,NaN
3,2020-02-25,97.0,4.0,NaN,2020,2,25,1,226.0,6.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.604318,0.066157,NaN
4,2020-02-26,93.0,5.0,NaN,2020,2,26,2,319.0,11.0,NaN,Italy,Europe,60461828.0,83.51,3.18,35220.084,1.538161,0.082697,NaN


## Writing data back to files

After completing your analysis and adding new columns, you should write the results back to a file. Otherwise, the data will be lost when the Jupyter notebook shuts down. Before writing to file, let us first create a data frame containing just the columns we wish to record.

#### TODO:

Save the dataframe to a CSV file. Call the file results.csv

In [52]:
merged_df.to_csv('results.csv', index = False)

To write the data from the data frame into a file, we can use the `to_csv` function. 